# Representing networks

## Representing networks mathematically

The most common representation of a network in a mathematical sense is an **adjacency matrix**. For a network with $n$ nodes, an adjacency matrix is an $n \times n$ matrix, which we'll often denote $A$. If there is an edge between node $i$ and node $j$, then there is some nonzero element at the $i, j$-th element of $A$, $a_{ij}$. 

For an **unweighted network**, the convention is for the matrix to have a "1" anywhere there is an edge, and a "0" otherwise. One example of an unweighted network is a social network, where each node is a person. We could say that there is an edge between two people if they are friends on Facebook, and there is not an edge otherwise.

For a **weighted network**, $a_{ij}$ will have a nonzero value representing the weight associated with that edge. For an example of a weighted network, we could consider another social network formed from email communications. We could say that there is an edge between any two people who emailed each other during a given time period, but we could further say that the edge weight is the *number of emails* between them.

For a **directed network**, 

## Representing networks in code